In [1]:
import pickle as pkl
import numpy as np

path_to_file = 'data/mimic3/singleDX.txt'

inv_idx = {}
# inv_map = {}

with open(path_to_file,'r') as f:
	data = f.read()

data = data.split('\n')

# inv_name = {}
var_idx_1 = 0
var_idx_2 = 0
# li = []
iidx = -1

for idx in range(0,len(data)):
	var_idx_2 = idx
	value = []
	key = ""
	if data[var_idx_2] == "":

		iidx = iidx + 1
		key = data[var_idx_1].split(" ")[0]
# 		inv_name[iidx] = ' '.join(data[var_idx_1].split(" ")[1:])

# 		inv_map[iidx] = key

		#extract the data between var_idx 1 and 2
		for i in range(var_idx_1+1, var_idx_2):
			l = [x for x in data[i].split(" ") if not x == '']
			
			# adding inverse lookup between the icd-code and collapsed heirarchy
			for x in l : 
				inv_idx[x] = iidx

# 			value = value + l
		
		#hierarchy[key] = value

		#set the new var_idx
		var_idx_1 = var_idx_2 + 1

# now pickle the hierarchy
# fpkl = open('inv_idx','wb')
# pkl.dump(inv_idx, fpkl)
# fpkl.close()


In [2]:
import pandas as pd
df = pd.read_csv('data/mimic3/ccs_multi_dx_tool_2015.csv')
td = df[['CODE','LVL','\'LVL2\'','\'LVL3\'','\'LVL4\'']]

td = td.values
for i in range(0,len(td)):
    for j in range(0, len(td[i])):
        td[i][j] = td[i][j].replace('\'','')
        td[i][j] = td[i][j].replace(' ','')
        


In [3]:
#Creating the hierarchy.
vmap = {}
for i in range(0, len(td)):
    ccs = inv_idx[td[i][0]]
    if ccs in vmap:
        continue
    else:
        vmap[ccs] = td[i][1:]



In [4]:
def common_anc(li,lj):
    ans_idx = 3
    for idx in range(0,4):
        if li[idx] != lj[idx]:
            ans_idx = idx - 1
            break
    return ans_idx

def hierarchical_distance(i,j):
    li = vmap[i]
    lj = vmap[j] 
    

    
    # idx : {-1, 0, 1, }
    anc = common_anc(li, lj)
    dist = 0
    
    if anc == -1 :
        dist = 2
        anc = 0
    else:
        dist = 0
    
#     for idx in range(anc, 4):
        
#         if li[idx] != '':
#             dist += 1
#         elif lj[idx] != '':
#             dist += 1
    dist += 2*(4-anc-1)
    
    return dist

In [5]:
# x,y=0,280
# print(common_anc(vmap[x],vmap[y]))
# hierarchical_distance(x,y)

In [6]:
vmap[283] = np.array(['19','','',''])

In [12]:
dist = [[0]*284]*284
for i in range(0,284):
    for j in range(i+1, 284):
        dist[i][j] = hierarchical_distance(i, j)
        
dist = np.array(dist)

In [15]:
#now pickle the hierarchy
hd = open('hier_dist','wb')
pkl.dump(dist, hd)
hd.close()